In [1]:
import hsup as _
import gymnasium as gym
from stable_baselines3 import PPO, DDPG, DQN
import torch
import numpy as np

from dataset.data import data
from dataset.simulation import model_indoor, model_sec_back_t

In [2]:
SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)


In [3]:
TOTAL_TIMESTEPS = int(1e3)
LOG_INTERVAL = 100
MODEL = DQN

env = gym.make(
    "hsup/HeatSupply-v0",
    max_episode_steps=24,

    data=data,
    model_indoor=model_indoor,
    model_sec_back_t=model_sec_back_t,
    discretize=True,
)

In [4]:
model = MODEL("MlpPolicy", env, verbose=1, tensorboard_log="./runs", seed=SEED)
model.learn(total_timesteps=TOTAL_TIMESTEPS, progress_bar=True, log_interval=LOG_INTERVAL)
model.save(f"checkpoints/{MODEL.__name__}-{TOTAL_TIMESTEPS}")


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./runs/DQN_1


Output()

/home/yanglsh/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:164: 
UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual 
type: float64
  logger.warn(

/home/yanglsh/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:188: 
UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")

/home/yanglsh/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/home/yanglsh/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [5]:
model = MODEL.load(f"checkpoints/{MODEL.__name__}-{TOTAL_TIMESTEPS}")

obs, _ = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, trunc, _ = env.step(action)
    if trunc or done:
        break
    
env.render()

sec_back_t_60min      51.311628
indoor_120min         25.301089
sec_supp_t_120min     60.624056
irradiance_60min     245.213258
outdoor_60min          2.416667
sec_back_t_120min     51.177764
irradiance           108.637439
outdoor                2.008333
outdoor_120min         2.266667
indoor_60min          25.235502
indoor_180min         25.195026
sec_back_t_180min     51.055040
irradiance_120min    378.676900
sec_supp_t_60min      60.624056
sec_supp_t_180min     60.624056
Name: 2022-02-02 15:00:00, dtype: float64
